In [13]:
import os
import random
import re
import pandas as pd
import numpy as np
import string


from gensim import models
from collections import defaultdict
from gensim import corpora
 
from gensim import similarities 
from sklearn.feature_extraction.text import CountVectorizer

import nltk

import pprint

STOPLIST = list(nltk.corpus.stopwords.words('english'))

# Import data

In [14]:
df = pd.read_csv("chunked_meetings.csv", sep = "|")
print(len(df))
df.sample()

7729


,document_id,leaid,state,district,document_year,document_month,document_day,text,chunk_number,chunk_id
4223,INDIANA_LINTON-STOCKTON SCHOOL CORPORATION_202...,1805910.0,INDIANA,LINTON-STOCKTON SCHOOL CORPORATION,2021.0,10.0,18.0,accept the recommendation and Mrs. Elliott sec...,2,INDIANA_LINTON-STOCKTON SCHOOL CORPORATION_202...


# Clean and process text

In [15]:
def process_text(
    text: str,
    lower_case: bool = True,
    remove_punct: bool = True,
    remove_stopwords: bool = False,
    lemma: bool = False,
    string_or_list: str = "str",
):
    tokens = nltk.word_tokenize(text)

    if lower_case:
        tokens = [token.lower() if token.isalpha() else token for token in tokens]

    if remove_punct:
        tokens = [token for token in tokens if token.isalpha()]

    if remove_stopwords:
        tokens = [token for token in tokens if not token in STOPLIST]

    if lemma:
        tokens = [nltk.wordnet.WordNetLemmatizer().lemmatize(token) for token in tokens]

    if string_or_list != "list":
        doc = " ".join(tokens)
    else:
        doc = tokens

    return doc

In [16]:
df["tokens"] = df.text.apply(process_text, remove_stopwords = False, string_or_list = "list")
df.sample()

,document_id,leaid,state,district,document_year,document_month,document_day,text,chunk_number,chunk_id,tokens
2659,MONTANA_BOZEMAN ELEM_2021_4_12_1,3004560.0,MONTANA,BOZEMAN ELEM,2021.0,4.0,12.0,"Ella Dietrich, Sierra DuCharme, Sara Guillen, ...",5,MONTANA_BOZEMAN ELEM_2021_4_12_1chunk5,"[ella, dietrich, sierra, ducharme, sara, guill..."


# Create a list of tokenized documents, called docs_list

In [17]:
docs_list = list(df.tokens)
# docs_list[0]

# Bi Grams & Tri Grams

The Phrases class from Gensim can be used to identify bigrams (two words that co-occur). 

Key parameters include: 

*min_count* set sets the minimum frequency threshold for a bigram to be considered (default = 5).
*delimeter* describes how bigrams will be combined into one token (default = "_")

*connector_words* allows you to optionally provide a list of connector words that are included in the bigram. For example, "of" would allow "bank of america" to be included. Default is no connector words. A simple option is setting connector_words=phrases.ENGLISH_CONNECTOR_WORDS

The resulting object from Phrases is a trained model (really, list of bigrams) that can be used to transform new documents by combining common phrases into single tokens.

In [18]:
from gensim.models import Phrases
from gensim.models import phrases


bigram_model = Phrases(docs_list, min_count=20, delimiter = "_", connector_words = phrases.ENGLISH_CONNECTOR_WORDS)

# Print bigram in document
for token in bigram_model[docs_list[10]]:
    if "_" in token:
        print(token)

call_to_order
roll_call
pledge_of_allegiance
action_items
memorandum_of_understanding
tuscaloosa_county
board_of_education
not_to_exceed
lowest_responsible
lowest_responsible
listed_below
agreement_between
tuscaloosa_county
delegate_assembly
consent_items
contract_between
elementary_school
fifth_grade
field_trip
contract_between
elementary_school
contract_between
elementary_school
contract_between
elementary_school


In [19]:
## Replace each bigram with a unigram delimited by a _
def replace_bigrams_with_phrases(token_list):
    return bigram_model[token_list]

df["tokens_phrases"] = df.tokens.apply(replace_bigrams_with_phrases)
docs_list = list(df.tokens_phrases)

In [20]:
trigram_model = Phrases(docs_list, min_count=10, delimiter = "_", connector_words = phrases.ENGLISH_CONNECTOR_WORDS)

# Print bigram in document
for token in trigram_model[docs_list[6]]:
    if "_" in token:
        print(token)

does_not
discriminate_on_the_basis
race_color
national_origin
these_policies
th_street
district_office
schuyler_ne
questions_regarding
career_and_technical_education
department_of_education
phone_fax
schuyler_community_schools
de_la
district_office
th_street
e_l
full_day
schuyler_community_schools
start_date


In [21]:
def replace_trigrams_with_phrases(token_list):
    return trigram_model[token_list]
df["tokens_phrases"] = df.tokens_phrases.apply(replace_trigrams_with_phrases)
docs_list = list(df.tokens_phrases)

# Removing single instances of letters

In [22]:
def remove_single_letters(token_list):
    return [token for token in token_list if len(token) > 1]

df["tokens_phrases"] = df.tokens_phrases.apply(remove_single_letters)

# Now Remove Stop Words

In [23]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Define a function to remove stop words
def remove_stop_words(token_list):
    return [token for token in token_list if token not in stop_words]

# Apply the function to remove stop words from your 'tokens_phrases' column
df['tokens_phrases'] = df.tokens_phrases.apply(remove_stop_words)

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


# Create gensim formatted corpus

Instead of using CountVectorizer to create a document-term matrix, gensim requires a special module specific format of a document term matrix. 

The Dictionary class generates a unique integer id for each unique token in the input list of documents. It then stores the mapping between the tokens and their ids in a dictionary-like object, which can be accessed using the token2id attribute.

In [24]:
from gensim.corpora import Dictionary

gensim_dictionary = Dictionary(df.tokens_phrases)
gensim_dictionary.token2id

{'accounts': 0,
 'activities': 1,
 'activity': 2,
 'activity_account': 3,
 'activity_fund': 4,
 'ad': 5,
 'administrators': 6,
 'advertisement': 7,
 'any_other': 8,
 'applicants': 9,
 'attend': 10,
 'board_room': 11,
 'book': 12,
 'box': 13,
 'building': 14,
 'calendar': 15,
 'cards': 16,
 'charge': 17,
 'comp': 18,
 'concession': 19,
 'contracts': 20,
 'conversations': 21,
 'custodial': 22,
 'directors': 23,
 'discuss': 24,
 'discussion': 25,
 'discussion_items': 26,
 'district': 27,
 'employees': 28,
 'essers': 29,
 'etc': 30,
 'event': 31,
 'floor': 32,
 'following': 33,
 'food_service': 34,
 'gate': 35,
 'get': 36,
 'gibbons': 37,
 'if_you_are': 38,
 'iii': 39,
 'introductions': 40,
 'involves': 41,
 'jamie': 42,
 'leadership_team': 43,
 'make_sure': 44,
 'management': 45,
 'managing': 46,
 'meeting': 47,
 'monday_july': 48,
 'money': 49,
 'mundil': 50,
 'ne': 51,
 'new': 52,
 'new_hires': 53,
 'newspaper': 54,
 'one': 55,
 'open': 56,
 'opening': 57,
 'options': 58,
 'please': 59,

We can limit the token dictionary to only include tokens that are present a certain number of times.

The filter_extremes() method is used to remove tokens from the dictionary that are either too rare or too common. It takes two arguments:

*no_below*: an integer that specifies the minimum number of documents that a token must appear in to be included in the dictionary (i.e., W). Tokens that appear in fewer than no_below documents are removed.

*no_above*: a float that specifies the maximum fraction of documents that a token can appear in to be included in the dictionary. Tokens that appear in more than no_above of the documents are removed.

In [25]:
gensim_dictionary.filter_extremes(no_below=20, no_above=0.65)

# Creating BOW & TF-IDF representations

Once the dictionary is created, it can be used to convert a document from a list of tokens to a bag-of-words representation, which is a sparse vector where each element represents the count of a particular token in the document. This is done using the doc2bow() method of the Dictionary class.

In [26]:
from gensim.models import TfidfModel

# Step 1: Create a list of BOW representations for the corpus
corpus_bow = []
for doc in docs_list:
    corpus_bow.append(gensim_dictionary.doc2bow(doc))

# Step 2: Train the TF-IDF model using the BoW corpus
tfidf_model = TfidfModel(corpus_bow)

# Step 3: Transform the BoW corpus into a TF-IDF corpus using the trained TF-IDF model
corpus_tfidf = tfidf_model[corpus_bow]

print("BoW Representation of the first document:", corpus_bow[0])
print("TF-IDF Representation of the first document:", list(corpus_tfidf[0]))

BoW Representation of the first document: [(0, 1), (1, 2), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 3), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 3), (22, 2), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 7), (41, 1), (42, 2), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 2), (55, 1), (56, 1), (57, 3), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 6), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 4), (71, 3), (72, 2), (73, 1), (74, 1), (75, 3), (76, 3), (77, 3), (78, 1), (79, 1), (80, 3), (81, 2), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1)]
TF-IDF Representation of the first document: [(0, 0.07341136143576983), (1, 0.09502698293416723), (2, 0.11894296431064792), (3, 0.

# Topic Modeling

# Train

We can use gensim.models.LdaModel to train an LDA model. 

Key parameters include:

*corpus*: list of documents represented using gensim dictionary doc-term matrix

*id2word*: a dictionary that maps integer word ids to the corresponding word strings. This is used to convert the bag-of-words and TF-IDF vectors back to human-readable text.

*num_topics*: the number of topics to learn from the corpus


In [27]:
from gensim.models import LdaModel

NUM_TOPICS_50 = 50
NUM_TOPICS_75 = 75
NUM_TOPICS_100 = 100

RANDOM_STATE = 5643

temp = gensim_dictionary[0] # required to initialize
word_id = gensim_dictionary.id2token

# Model Descriptions

- LDA Model 1 is trained with 50 topics and BOW representation of the documents
- LDA Model 2 is trained with 50 topics and BOW representation of the documents
- LDA Model 3 is trained with 75 topics and BOW representation of the documents
- LDA Model 4 is trained with 75 topics and TF-IDF representation of the documents
- LDA Model 5 is trained with 100 topics and TF-IDF representation of the documents
- LDA Model 6 is trained with 100 topics and TF-IDF representation of the documents

# BOW Models

In [28]:
lda_model_1 = LdaModel(
    corpus=corpus_bow,
    id2word=word_id,
    num_topics=NUM_TOPICS_50,
    random_state = RANDOM_STATE,
)

lda_model_2 = LdaModel(
    corpus=corpus_bow,
    id2word=word_id,
    num_topics=NUM_TOPICS_75,
    random_state = RANDOM_STATE,
)

lda_model_3 = LdaModel(
    corpus=corpus_bow,
    id2word=word_id,
    num_topics=NUM_TOPICS_100,
    random_state = RANDOM_STATE,
)

# TF-IDF Models

In [29]:
lda_model_4 = LdaModel(
    corpus=corpus_tfidf,
    id2word=word_id,
    num_topics=NUM_TOPICS_50,
    random_state = RANDOM_STATE,
)

lda_model_5 = LdaModel(
    corpus=corpus_tfidf,
    id2word=word_id,
    num_topics=NUM_TOPICS_75,
    random_state = RANDOM_STATE,
)

lda_model_6 = LdaModel(
    corpus=corpus_tfidf,
    id2word=word_id,
    num_topics=NUM_TOPICS_100,
    random_state = RANDOM_STATE,
)

After training the model, we can view the words that are important for each topic using the show_topics method.

Key parameters include:
    

*num_topics*: The number of topics to be selected, if -1 - all topics will be in result.

*num_words*: The number of words to be included per topics.

*formatted*: The formatted=False argument indicates that the method should return a list of (word, probability) pairs for each word in the topic, rather than a formatted string.


# Model 1 with 50 topics and BOW representation of the documents

In [30]:
lda_model_1.show_topics(num_topics = -1, num_words= 10, formatted = False)[0]

(0,
 [('records', 0.029021408),
  ('program', 0.014085579),
  ('school', 0.011822059),
  ('university', 0.009700679),
  ('disposition', 0.009389855),
  ('records_retention', 0.0065970924),
  ('services', 0.0059656654),
  ('senior_high_school', 0.005943884),
  ('disposed', 0.005894095),
  ('school_counselors', 0.0055533773)])

# Model 2 with 50 topics and BOW representation of the documents

In [31]:
lda_model_2.show_topics(num_topics = -1, num_words= 10, formatted = False)[0]

(0,
 [('school', 0.011963719),
  ('hardin', 0.011544713),
  ('license', 0.010524992),
  ('be_granted', 0.010330215),
  ('red', 0.009486529),
  ('cases', 0.009424991),
  ('acting', 0.008764949),
  ('positions', 0.007855811),
  ('public', 0.0078251725),
  ('program', 0.0065753376)])

# Model 3 with 75 topics and BOW representation of the documents

In [32]:
lda_model_3.show_topics(num_topics = -1, num_words= 10, formatted = False)[0]

(0,
 [('program', 0.012152751),
  ('license', 0.011158229),
  ('temporary', 0.010714946),
  ('increase', 0.0103770895),
  ('agency', 0.00971529),
  ('records', 0.009712331),
  ('university', 0.0084626665),
  ('renewal', 0.008063529),
  ('school', 0.007848108),
  ('district', 0.007722864)])

# Model 4 with 75 topics and TF-IDF representation of the documents

In [33]:
lda_model_4.show_topics(num_topics = -1, num_words= 10, formatted = False)[0]

(0,
 [('motion_unanimously_carried', 0.012804213),
  ('permanent_record', 0.012618397),
  ('extra_curricular', 0.008057409),
  ('tucker', 0.007540574),
  ('erickson', 0.0056745373),
  ('trainings', 0.0054784752),
  ('dunn', 0.0050404966),
  ('enrichment', 0.004779847),
  ('virtual', 0.0043417634),
  ('soil', 0.0042633316)])

# Model 5 with 100 topics and TF-IDF representation of the documents

In [34]:
lda_model_5.show_topics(num_topics = -1, num_words= 10, formatted = False)[0]

(0,
 [('member', 0.008603532),
  ('policy', 0.008278606),
  ('agenda_item', 0.006793281),
  ('executive_session', 0.0061175684),
  ('zoom', 0.0052946387),
  ('board', 0.0049904846),
  ('public', 0.004968534),
  ('checks', 0.004936207),
  ('total_amount', 0.004444237),
  ('boe', 0.004436045)])

# Model 6 with 100 topics and TF-IDF representation of the documents

In [35]:
lda_model_6.show_topics(num_topics = -1, num_words= 10, formatted = False)[0]

(0,
 [('motion_unanimously_carried', 0.021804113),
  ('instr', 0.012906829),
  ('ba_step', 0.0109865535),
  ('new_business', 0.010129189),
  ('nj', 0.009757953),
  ('erickson', 0.009587086),
  ('unfinished_business', 0.009351331),
  ('social_studies', 0.0093494225),
  ('public_comment_none', 0.009218349),
  ('grade', 0.008941581)])

# Coherence

In [36]:
from gensim.models.coherencemodel import CoherenceModel

def calculate_average_coherence(lda_model, corpus, gensim_dictionary, num_topics, topn=10):
    """
    Calculate the average topic coherence for a given LDA model.

    Parameters:
    - lda_model: The LDA model to evaluate.
    - corpus: The corpus used by the LDA model (BoW or TF-IDF).
    - gensim_dictionary: The Gensim dictionary of the corpus.
    - num_topics: The number of topics used in the LDA model.
    - topn: The number of top terms to consider for calculating coherence (default=10).

    Returns:
    - The average coherence score across all topics in the LDA model.
    """
    topic_coherence_scores = []
    for n in range(lda_model.num_topics):
        topn_terms = [term for term, _ in lda_model.show_topic(n, topn=topn)]
        coherence = CoherenceModel(topics=[topn_terms], corpus=corpus, dictionary=gensim_dictionary, coherence='u_mass', topn=topn)
        score = coherence.get_coherence()
        topic_coherence_scores.append(score)

    avg_coherence = sum(topic_coherence_scores) / num_topics
    return avg_coherence

In [37]:
mod_1_avg_coherence = calculate_average_coherence(lda_model_1, corpus_bow, gensim_dictionary, NUM_TOPICS_50)
mod_2_avg_coherence = calculate_average_coherence(lda_model_2, corpus_bow, gensim_dictionary, NUM_TOPICS_75)
mod_3_avg_coherence = calculate_average_coherence(lda_model_3, corpus_bow, gensim_dictionary, NUM_TOPICS_100)

mod_4_avg_coherence = calculate_average_coherence(lda_model_4, corpus_tfidf, gensim_dictionary, NUM_TOPICS_50)
mod_5_avg_coherence = calculate_average_coherence(lda_model_5, corpus_tfidf, gensim_dictionary, NUM_TOPICS_75)
mod_6_avg_coherence = calculate_average_coherence(lda_model_6, corpus_tfidf, gensim_dictionary, NUM_TOPICS_100)

# You can print the average coherence scores for each model to see the results
print("Model 1 Average Coherence:", mod_1_avg_coherence)
print("Model 2 Average Coherence:", mod_2_avg_coherence)
print("Model 3 Average Coherence:", mod_3_avg_coherence)
print("Model 4 Average Coherence:", mod_4_avg_coherence)
print("Model 5 Average Coherence:", mod_5_avg_coherence)
print("Model 6 Average Coherence:", mod_6_avg_coherence)

Model 1 Average Coherence: -3.3775469953314956
Model 2 Average Coherence: -3.7645702456361656
Model 3 Average Coherence: -4.676187474459707
Model 4 Average Coherence: -8.92425426798147
Model 5 Average Coherence: -10.613656592226308
Model 6 Average Coherence: -9.030562039997365


## Create dataframe of top words per topic for our best model: Model 2

In [38]:
list_of_topic_tables = []
for topic in lda_model_2.show_topics(
    num_topics=-1, num_words=10, formatted=False
):
    list_of_topic_tables.append(
        pd.DataFrame(
            data = topic[1],
            columns=["Word" + "_" + str(topic[0]), "Prob" + "_" + str(topic[0])],
        )
    )
list_of_topic_tables[0]

,Word_0,Prob_0
0,school,0.011964
1,hardin,0.011545
2,license,0.010525
3,be_granted,0.010330
4,red,0.009487
5,cases,0.009425
6,acting,0.008765
7,positions,0.007856
8,public,0.007825
9,program,0.006575


In [39]:
pd.set_option('display.max_columns', 500)

bigdf_mod2 = pd.concat(list_of_topic_tables, axis=1)
bigdf_mod2

,Word_0,Prob_0,Word_1,Prob_1,Word_2,Prob_2,Word_3,Prob_3,Word_4,Prob_4,Word_5,Prob_5,Word_6,Prob_6,Word_7,Prob_7,Word_8,Prob_8,Word_9,Prob_9,Word_10,Prob_10,Word_11,Prob_11,Word_12,Prob_12,Word_13,Prob_13,Word_14,Prob_14,Word_15,Prob_15,Word_16,Prob_16,Word_17,Prob_17,Word_18,Prob_18,Word_19,Prob_19,Word_20,Prob_20,Word_21,Prob_21,Word_22,Prob_22,Word_23,Prob_23,Word_24,Prob_24,Word_25,Prob_25,Word_26,Prob_26,Word_27,Prob_27,Word_28,Prob_28,Word_29,Prob_29,Word_30,Prob_30,Word_31,Prob_31,Word_32,Prob_32,Word_33,Prob_33,Word_34,Prob_34,Word_35,Prob_35,Word_36,Prob_36,Word_37,Prob_37,Word_38,Prob_38,Word_39,Prob_39,Word_40,Prob_40,Word_41,Prob_41,Word_42,Prob_42,Word_43,Prob_43,Word_44,Prob_44,Word_45,Prob_45,Word_46,Prob_46,Word_47,Prob_47,Word_48,Prob_48,Word_49,Prob_49,Word_50,Prob_50,Word_51,Prob_51,Word_52,Prob_52,Word_53,Prob_53,Word_54,Prob_54,Word_55,Prob_55,Word_56,Prob_56,Word_57,Prob_57,Word_58,Prob_58,Word_59,Prob_59,Word_60,Prob_60,Word_61,Prob_61,Word_62,Prob_62,Word_63,Prob_63,Word_64,Prob_64,Word_65,Prob_65,Word_66,Prob_66,Word_67,Prob_67,Word_68,Prob_68,Word_69,Prob_69,Word_70,Prob_70,Word_71,Prob_71,Word_72,Prob_72,Word_73,Prob_73,Word_74,Prob_74
0,school,0.011964,child,0.021755,students,0.016050,regular,0.045083,school_year,0.018263,fund,0.036625,school,0.015710,approval,0.044826,board,0.026196,school,0.021004,these_records,0.024086,board,0.032023,tax,0.018558,nicole,0.011723,salary,0.034185,instructional_assistant,0.013684,data,0.019795,project,0.014346,board_of_education,0.043032,district,0.023076,students,0.021544,teachers,0.019599,student,0.014854,school,0.016624,resolution,0.022471,records,0.017432,days,0.045864,coordinator,0.022273,motion,0.051037,february,0.044425,student,0.028726,students,0.010683,district,0.015635,students,0.037845,students,0.014479,district,0.023959,executive,0.028146,trip,0.019807,seconded,0.013310,school,0.021687,appointment,0.031905,club,0.013158,this_resolution,0.030680,board,0.014349,years,0.017433,action,0.082055,meeting,0.037003,approve,0.061100,bonds,0.032314,approve_the_recommendation,0.024564,president,0.020927,yes,0.090196,reports,0.014575,district,0.026036,board,0.029076,motion,0.056733,motion,0.038118,local,0.028219,files,0.019903,board_of_education,0.019257,school,0.022130,meeting,0.015554,stipend,0.021454,high_school,0.019678,revised,0.028358,election,0.056966,page,0.031333,chairperson,0.016317,board,0.025869,school,0.020138,board_member,0.022356,board,0.022897,board,0.020256,meeting,0.037204,board,0.017298
1,hardin,0.011545,board_of_education,0.015592,board,0.012738,seconded,0.023674,seconded,0.011883,total,0.018945,funds,0.012569,approved_the_resolution,0.041231,budget,0.017649,health,0.015129,record,0.019344,policy,0.028176,shs,0.015569,thomas,0.010239,districtwide,0.026968,district,0.011550,goals,0.017808,award,0.013891,pending,0.018105,application,0.014896,district,0.017473,grades,0.016767,students,0.012651,williams,0.016137,approve,0.019949,school,0.012999,student,0.033791,mhs,0.018387,board,0.030592,recommendation,0.026488,students,0.023650,district,0.008136,schedule,0.011757,stated_that,0.011058,school,0.007862,records,0.022766,se,0.017925,board,0.012256,board,0.012523,students,0.011625,teacher,0.028529,substitute,0.012829,section,0.028573,meeting,0.013041,september,0.016812,motion,0.043776,board,0.029246,seconded,0.031448,issuance,0.017721,double_click_to_return_to_agenda,0.023818,student,0.017549,trustee,0.056963,policy,0.013398,budget,0.009520,assistant,0.017530,approve,0.032863,effective,0.030687,grant,0.012016,records,0.016068,resolved_that,0.017996,student,0.013170,seconded,0.012142,recommend,0.015035,students,0.018527,replaces,0.022617,elections,0.045063,yes,0.030350,present,0.011892,students,0.012572,system,0.012248,school,0.020160,following,0.014112,policy,0.013193,board,0.022704,school,0.016548
2,license,0.010525,lists,0.009797,will_be,0.011487,approve,0.020779,motion_passed_unanimously,0.011241,supplies,0.018786,project,0.011460,carried_by_a_vote,0.0

In [40]:
topic_probs = []
for document in corpus_bow:
    document_topics = []
    for topic_prob in lda_model_2.get_document_topics(document, minimum_probability=0):
        document_topics.append(topic_prob[1])
    topic_probs.append(document_topics)
topic_probs[0]

[9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 0.06837408,
 0.11344579,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 0.048586573,
 9.599427e-05,
 9.599427e-05,
 0.045556676,
 0.059315577,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 0.06507122,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 0.029658299,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.599427e-05,
 9.59

In [41]:
topic_probs_df = pd.DataFrame(topic_probs)
topic_probs_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74
0,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.068374,0.113446,0.000096,0.000096,0.000096,0.000096,0.000096,0.048587,0.000096,0.000096,0.045557,0.059316,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.065071,0.000096,0.000096,0.000096,0.029658,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.045999,0.000096,0.142462,0.000096,0.152624,0.222403,0.000456
1,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.192150,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.060726,0.035805,0.000090,0.000090,0.000090,0.000090,0.000090,0.025824,0.000090,0.104165,0.000090,0.000090,0.000090,0.000090,0.024729,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.098874,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.159681,0.000090,0.292137,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090
2,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.178825,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.180272,0.000094,0.000094,0.000094,0.000094,0.000094,0.050073,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.035216,0.000094,0.000094,0.000094,0.000094,0.038953,0.100825,0.000094,0.000094,0.000094,0.000094,0.056760,0.000094,0.092708,0.000094,0.000094,0.000094,0.000094,0.000094,0.063557,0.000094,0.000094,0.013626,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.020246,0.000094,0.000094,0.024044,0.097666,0.041495,0.000094,0.000094,0.000094,0.000094,0.000094
3,0.000102,0.000102,0.000102,0.000102,0.000102,0.071785,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.683986,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.034273,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.093440,0.000102,0.000102,0.000102,0.000102,0.046863,0.000102,0.000102,0.062627,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102
4,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.039093,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.588911,0.009429,0.000113,0.000113,0.000113,0.000113,0.011354,0.000113,0.000113,0.084924,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.156068,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.084705,0.000113,0.000113,0.000113,0.000113,0.017940,0.000113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7724,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.024024,0.000125,

In [42]:
topic_probs_df.sum(axis = 1)

0       1.000000
1       1.000000
2       1.000001
3       0.999999
4       1.000000
          ...   
7724    1.000001
7725    1.000001
7726    1.000000
7727    1.000000
7728    1.000000
Length: 7729, dtype: float32

In [43]:
doc_topics = df.reset_index().merge(topic_probs_df, left_index=True, right_index=True)
doc_topics

,index,document_id,leaid,state,district,document_year,document_month,document_day,text,chunk_number,chunk_id,tokens,tokens_phrases,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74
0,0,NEBRASKA_SCHUYLER COMMUNITY SCHOOLS_2021_7_26_1,3176450.0,NEBRASKA,SCHUYLER COMMUNITY SCHOOLS,2021.0,7.0,26.0,Schuyler Community Schools SAA Weekly Meeting/...,1,NEBRASKA_SCHUYLER COMMUNITY SCHOOLS_2021_7_26_...,"[schuyler, community, schools, saa, weekly, te...","[schuyler_community_schools, saa, weekly, team...",0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.068374,0.113446,0.000096,0.000096,0.000096,0.000096,0.000096,0.048587,0.000096,0.000096,0.045557,0.059316,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.065071,0.000096,0.000096,0.000096,0.029658,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.045999,0.000096,0.142462,0.000096,0.152624,0.222403,0.000456
1,1,NEBRASKA_SCHUYLER COMMUNITY SCHOOLS_2021_7_26_1,3176450.0,NEBRASKA,SCHUYLER COMMUNITY SCHOOLS,2021.0,7.0,26.0,positions open at this time? 5. Friday Substit...,2,NEBRASKA_SCHUYLER COMMUNITY SCHOOLS_2021_7_26_...,"[positions, open, at, this, time, friday, subs...","[positions, open, this_time, friday, substitut...",0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.192150,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.060726,0.035805,0.000090,0.000090,0.000090,0.000090,0.000090,0.025824,0.000090,0.104165,0.000090,0.000090,0.000090,0.000090,0.024729,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.098874,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.159681,0.000090,0.292137,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090,0.000090
2,2,NEBRASKA_SCHUYLER COMMUNITY SCHOOLS_2021_7_26_1,3176450.0,NEBRASKA,SCHUYLER COMMUNITY SCHOOLS,2021.0,7.0,26.0,Town to Fisher's: 11 Miles Town to Richland: 2...,3,NEBRASKA_SCHUYLER COMMUNITY SCHOOLS_2021_7_26_...,"[town, to, fisher, miles, town, to, richland, ...","[town, fisher, miles, town, richland, miles, f...",0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.178825,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.180272,0.000094,0.000094,0.000094,0.000094,0.000094,0.050073,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.035216,0.000094,0.000094,0.000094,0.000094,0.038953,0.100825,0.000094,0.000094,0.000094,0.000094,0.056760,0.000094,0.092708,0.000094,0.000094,0.000094,0.000094,0.000094,0.063557,0.000094,0.000094,0.013626,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.020246,0.000094,0.000094,0.024044,0.097666,0.041495,0.000094,0.000094,0.000094,0.000094,0.000094
3,3,NEBRASKA_SCHUYLER COMMUNITY SCHOOLS_2021_7_26_1,3176450.0,NEBRASKA,SCHUYLER COMMUNITY SCHOOLS,2021.0,7.0,26.0,each week. 21. 403B Elective and Non-Elective ...,4,NEBRASKA_SCHUYLER COMMUNITY SCHOOLS_2021_7_26_...,"[each, week, elective, and, contributions, all...","[week, elective, contributions, employees, opt...",0.000102,0.000102,0.000102,0.000102,0.000102,0.071785,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.683986,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.000102,0.034273,0.000102,0.000102,0.000102,0.000102,0.00010

# Topic 55

In [47]:
doc_topics.sort_values(by = 55, ascending = False)

,index,document_id,leaid,state,district,document_year,document_month,document_day,text,chunk_number,chunk_id,tokens,tokens_phrases,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74
1364,1364,KENTUCKY_BOYD COUNTY_2020_11_17_2,2100620.0,KENTUCKY,BOYD COUNTY,2020.0,11.0,17.0,for the Boyd County High School Phase IV Athle...,12,KENTUCKY_BOYD COUNTY_2020_11_17_2chunk12,"[for, the, boyd, county, high, school, phase, ...","[boyd_county_high_school, phase_iv_athletics_p...",0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.976504,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318
1363,1363,KENTUCKY_BOYD COUNTY_2020_11_17_2,2100620.0,KENTUCKY,BOYD COUNTY,2020.0,11.0,17.0,unused purchase order amount) for the Boyd Cou...,11,KENTUCKY_BOYD COUNTY_2020_11_17_2chunk11,"[unused, purchase, order, amount, for, the, bo...","[unused_purchase_order, amount, boyd_county_hi...",0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.974697,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342,0.000342
276,276,WISCONSIN_MONONA GROVE SCHOOL DISTRICT_2021_10...,5509810.0,WISCONSIN,MONONA GROVE SCHOOL DISTRICT,2021.0,10.0,27.0,MINUTES : Meeting of the Monona Grove School D...,1,WISCONSIN_MONONA GROVE SCHOOL DISTRICT_2021_10...,"[minutes, meeting, of, the, monona, grove, sch...","[minutes, meeting, monona_grove_school_distric...",0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.035784,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.955366,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121
1508,1508,IOWA_WAUKEE COMM SCHOOL DISTRICT_2022_7_11_2,1930510.0,IOWA,WAUKEE COMM SCHOOL DISTRICT,2022.0,7.0,11.0,for the Waukee Waterford Elementary School Pro...,5,IOWA_WAUKEE COMM SCHOOL DISTRICT_2022_7_11_2ch...,"[for, the, waukee, waterford, elementary, scho...","[waukee, waterford, elementary_school, project...",0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.038459,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.000252,0.0

In [50]:
doc_topics.iloc[doc_topics[55].idxmax()].text

"for the Boyd County High School Phase IV Athletics project for a credit in the amount of Seventy-five cents ($.75) as presented by Sherman-Carter-Barnhart Architects; (See Attachment) Order #0096 - Motion Passed: Consider Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 16 (Credit unused purchase order amount) for the Boyd County High School Phase IV Athletics project for a credit in the amount of Seventy-five cents ($.75) as presented by Sherman-Carter-Barnhart Architects passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 12.14. Approve Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 24 (Credit unused purchase order amount) for the Boyd County High School Phase IV Athletics project for a credit in the amount of One thousand eight hundred forty-seven dollars and Fift

"for the Boyd County High School Phase IV Athletics project for a credit in the amount of Seventy-five cents ($.75) as presented by Sherman-Carter-Barnhart Architects; (See Attachment) Order #0096 - Motion Passed: Consider Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 16 (Credit unused purchase order amount) for the Boyd County High School Phase IV Athletics project for a credit in the amount of Seventy-five cents ($.75) as presented by Sherman-Carter-Barnhart Architects passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 12.14. Approve Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 24 (Credit unused purchase order amount) for the Boyd County High School Phase IV Athletics project for a credit in the amount of One thousand eight hundred forty-seven dollars and Fifty cents ($1,847.50) as presented by Sherman-Carter-Barnhart Architects; (See Attachment) Order #0097 - Motion Passed: Consider Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 24 (Credit unused purchase order amount) for the Boyd County High School Phase IV Athletics project for a credit in the amount of One thousand eight hundred forty-seven P a g e 56 dollars and Fifty cents ($1,847.50) as presented by Sherman-Carter-Barnhart Architects passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 12.15. Approve Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 26 (Credit unused purchase order amount) for the Boyd County High School Phase IV Athletics project for a credit in the amount of"

In [51]:
doc_topics.iloc[doc_topics[55].idxmax() + 1].text

"Seven hundred forty-five dollars and Ninety-six cents ($745.96) as presented by Sherman-Carter-Barnhart Architects; (See Attachments) Order #0098 - Motion Passed: Consider Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 26 (Credit unused purchase order amount) for the Boyd County High School Phase IV Athletics project for a credit in the amount of Seven hundred forty-five dollars and Ninety-six cents ($745.96) as presented by Sherman-Carter-Barnhart Architects passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 12.16. Approve Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 27 (Credit unused purchase order amount) for the Boyd County High School Phase IV Athletics project for a credit in the amount of Seven dollars and Ninety-five cents ($7.95) as presented by Sherman

"Seven hundred forty-five dollars and Ninety-six cents ($745.96) as presented by Sherman-Carter-Barnhart Architects; (See Attachments) Order #0098 - Motion Passed: Consider Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 26 (Credit unused purchase order amount) for the Boyd County High School Phase IV Athletics project for a credit in the amount of Seven hundred forty-five dollars and Ninety-six cents ($745.96) as presented by Sherman-Carter-Barnhart Architects passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 12.16. Approve Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 27 (Credit unused purchase order amount) for the Boyd County High School Phase IV Athletics project for a credit in the amount of Seven dollars and Ninety-five cents ($7.95) as presented by Sherman-Carter- Barnhart Architects; (See Attachment) Order #0099 - Motion Passed: Consider Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 27 (Credit unused purchase order amount) for the Boyd County High School Phase IV Athletics project for a credit in the amount of Seven dollars and Ninety-five cents ($7.95) as presented by Sherman-Carter-Barnhart Architects passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 12.17. Approve Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 14 (Material cost for pole lights/4 poles, 6 heads) for the Boyd County High School Phase IV Athletics project for an increase in the amount of Nine thousand two hundred twenty-one dollars and Sixty cents ($9,221.60) as presented by Sherman-Carter-Barnhart Architects, (See"

In [52]:
doc_topics.iloc[doc_topics[55].idxmax() + 2].text

"Attachment) Order #00100 - Motion Passed: Consider Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 27 (Material cost for pole lights/4 poles, 6 heads) for the Boyd County High School Phase IV Athletics project for an increase in the amount of Nine thousand two hundred twenty-one dollars and Sixty cents ($9,221.60) as presented by Sherman-Carter-Barnhart Architects passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes P a g e 57 Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 12.18. Approve Superintendent's recommendation for Board members to attend the Kentucky School Boards' Association 2020 Winter Symposium December 3 through 5, 2020 at the Marriott Downtown in Louisville, Kentucky; Rationale: Those attending will be: Tammy Pruitt, Mindy Stanley, Randy Stapleton, Jr. Order #00101 - Motion Passed: Consider Superintendent's recommendation to approve Board m

"Attachment) Order #00100 - Motion Passed: Consider Superintendent's recommendation to approve the Direct Purchase Order Change Order No. DPO 27 (Material cost for pole lights/4 poles, 6 heads) for the Boyd County High School Phase IV Athletics project for an increase in the amount of Nine thousand two hundred twenty-one dollars and Sixty cents ($9,221.60) as presented by Sherman-Carter-Barnhart Architects passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes P a g e 57 Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 12.18. Approve Superintendent's recommendation for Board members to attend the Kentucky School Boards' Association 2020 Winter Symposium December 3 through 5, 2020 at the Marriott Downtown in Louisville, Kentucky; Rationale: Those attending will be: Tammy Pruitt, Mindy Stanley, Randy Stapleton, Jr. Order #00101 - Motion Passed: Consider Superintendent's recommendation to approve Board members' attending the Kentucky School Boards' Association 2020 Winter Symposium December 3 through 5, 2020 at the Marriott Downtown in Louisville, Kentucky passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 12.19. Approve Superintendent's recommendation to approve the request from Cindy Pontier to advertise and accept bids for the Digital Data Service for Hub (Heritage), Summit, Boyd County Middle, Catlettsburg, Central Office, Ponderosa Elementary, Ramey-Estep High and Boyd County High locations as our current contracts are expiring June 30, 2021; (See Attachment) Order #00102 - Motion Passed: Approve Superintendent's recommendation to approve the request from Cindy Pontier to advertise and accept bids for the Digital Data Service for Hub (Heritage), Summit, Boyd County Middle, Catlettsburg, Central Office, Ponderosa Elementary, Ramey-Estep"

In [53]:
doc_topics.iloc[doc_topics[55].idxmax() + 3].text

"High and Boyd County High locations as our current contracts are expiring June 30, 2021 passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 12.20. Approve Superintendent's recommendation to approve request of Cindy Pontier to advertise and accept bids for the renovation/replacement/upgrade of wired and/or wireless data networks at Boyd County High, Ramey-Estep High, Cannonsburg Elementary, Catlettsburg Elementary, Ponderosa Elementary and Summit Elementary; (See Attachment) Order #00103 - Motion Passed: Consider Superintendent's recommendation to approve request of Cindy Pontier to advertise and accept bids for the renovation/replacement/upgrade of wired and/or wireless data networks at Boyd County High, Ramey-Estep High, Cannonsburg Elementary, Catlettsburg Elementary, Ponderosa Elementary and Summit Elementary passed with a motion by Ms. Tammy Pru

"High and Boyd County High locations as our current contracts are expiring June 30, 2021 passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 12.20. Approve Superintendent's recommendation to approve request of Cindy Pontier to advertise and accept bids for the renovation/replacement/upgrade of wired and/or wireless data networks at Boyd County High, Ramey-Estep High, Cannonsburg Elementary, Catlettsburg Elementary, Ponderosa Elementary and Summit Elementary; (See Attachment) Order #00103 - Motion Passed: Consider Superintendent's recommendation to approve request of Cindy Pontier to advertise and accept bids for the renovation/replacement/upgrade of wired and/or wireless data networks at Boyd County High, Ramey-Estep High, Cannonsburg Elementary, Catlettsburg Elementary, Ponderosa Elementary and Summit Elementary passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 12.21. Approve Superintendent's recommendation to approve and accept the School Facilities Construction Commission (SFCC) 2021 KETS (Kentucky Education Technology System) first Offer of Assistance in the amount of $20,202 (Twenty P a g e 58 thousand two hundred and two dollars) and to match it with money from the General Fund; (See Attachment) Rationale: Pursuant to KRS 157.655-157.660 and 750 KAR 2:010 Acceptance of this offer of assistance commits the Board of Education to follow statutes and regulations of the SFCC and KDE governing the Education Technology Funding Program. This offer is based on the 2020 statewide ADA (Average Daily Attendance) figures. Fund 2, Project 162G EdTech account Order #00104 - Motion Passed: Consider Superintendent's recommendation to approve and accept the School Facilities Construction Commission (SFCC) 2021 KETS (Kentucky"

In [54]:
doc_topics.iloc[doc_topics[55].idxmax() + 4].text

"Education Technology System) first Offer of Assistance in the amount of $20,202 (Twenty thousand two hundred and two dollars) and to match it with money from the General Fund passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 13. Consent Items Order #00105 - Motion Passed: Consider superintendent's recommendation to approve consent items as listed passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 13.1. Consider Financial Report - October 2020 S Burchett 13.2. Consider payment of bills - November 2020 Helen Campbell 13.3. Consider Minutes of a regular board meeting of October27, 2020 (See Attachment) 13.4. Consider minutes of the School Based Decision Making Councils as listed below: Boyd County Middle October 1

"Education Technology System) first Offer of Assistance in the amount of $20,202 (Twenty thousand two hundred and two dollars) and to match it with money from the General Fund passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 13. Consent Items Order #00105 - Motion Passed: Consider superintendent's recommendation to approve consent items as listed passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt Yes Mrs. Mindy Stanley Yes Mr. Randall Stapleton, Jr Yes Mrs. Lisa Wallin Yes 13.1. Consider Financial Report - October 2020 S Burchett 13.2. Consider payment of bills - November 2020 Helen Campbell 13.3. Consider Minutes of a regular board meeting of October27, 2020 (See Attachment) 13.4. Consider minutes of the School Based Decision Making Councils as listed below: Boyd County Middle October 14, 2020 Boyd County Middle Spec October 30, 2020 Catlettsburg Elementary October 21, 2020 14. Acknowledge Personnel Actions - November 2020 Chairperson acknowledged Personnel Actions. No questions or discussion. 15. Focus of Business/Topics for Next Meeting - December 22 Dialogue: 1. Job responsibilities/duties of Administrative positions (Superintendent Boblett) 2. Board Member Orientation for new members (Superintendent Boblett) Reports: 1. Revenue Projections (Scott Burchett) 2. Mr. Boblett stated he would be having the New Board Member Boot Camp with Lisa as he has done in the past but will look at a later date due to COVID, maybe January or February. 16. Adjourned at 6:53 PM Order #00106 - Motion Passed: Adjournment passed with a motion by Ms. Tammy Pruitt and a second by Mrs. Mindy Stanley. Ms. Linda Day Yes Ms. Tammy Pruitt"

# Topic 14

In [61]:
doc_topics.sort_values(by = 14, ascending = False)

,index,document_id,leaid,state,district,document_year,document_month,document_day,text,chunk_number,chunk_id,tokens,tokens_phrases,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74
648,648,NEW JERSEY_BERNARDS TOWNSHIP SCHOOL DISTRICT_2...,3401650.0,NEW JERSEY,BERNARDS TOWNSHIP SCHOOL DISTRICT,2021.0,1.0,25.0,Special Education Teacher Cedar Hill School Sp...,18,NEW JERSEY_BERNARDS TOWNSHIP SCHOOL DISTRICT_2...,"[special, education, teacher, cedar, hill, sch...","[special_education_teacher, cedar_hill_school,...",0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.988389,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157,0.000157
646,646,NEW JERSEY_BERNARDS TOWNSHIP SCHOOL DISTRICT_2...,3401650.0,NEW JERSEY,BERNARDS TOWNSHIP SCHOOL DISTRICT,2021.0,1.0,25.0,per month Extra Section Algebra II RHS Decembe...,16,NEW JERSEY_BERNARDS TOWNSHIP SCHOOL DISTRICT_2...,"[per, month, extra, section, algebra, ii, rhs,...","[per_month, extra_section, algebra, ii, rhs, d...",0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.983552,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222,0.000222
650,650,NEW JERSEY_BERNARDS TOWNSHIP SCHOOL DISTRICT_2...,3401650.0,NEW JERSEY,BERNARDS TOWNSHIP SCHOOL DISTRICT,2021.0,1.0,25.0,of the Cricut part 1 new 4 $350 Courtney Slack...,20,NEW JERSEY_BERNARDS TOWNSHIP SCHOOL DISTRICT_2...,"[of, the, cricut, part, new, courtney, slack, ...","[cricut, part, new, courtney, slack, abcs, cri...",0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.923771,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.062516,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188,0.000188
2542,2542,NEBRASKA_CHASE COUNTY SCHOOLS_2022_1_11_1,3100163.0,NEBRASKA,CHASE COUNTY SCHOOLS,2022.0,1.0,11.0,"0.00 1,244.90 4,976.33 13,000.00 8,023.67 01-2...",20,NEBRASKA_CHASE COUNTY SCHOOLS_2022_1_11_1chunk20,"[special, education, instructional, programs, ...","[special_education_instructional_programs, sch...",0.000460,0.000460,0.000460,0.000460,0.000460,0.042667,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.923768,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460,0.000460

In [62]:
print(doc_topics.iloc[doc_topics[14].idxmax()].text)
print(doc_topics.iloc[doc_topics[14].idxmax() + 1].text)
print(doc_topics.iloc[doc_topics[14].idxmax() + 2].text)
print(doc_topics.iloc[doc_topics[14].idxmax() + 3].text)
print(doc_topics.iloc[doc_topics[14].idxmax() + 4].text)

Special Education Teacher Cedar Hill School Special Education Teacher Liberty Corner School Page 254 salary of Step 1-4 MA $59,355 effective March 22, 2021 through June 30, 2021 as a leave replacement for M. Harvey. Salary to be prorated to reflect start date. 16) The Bernards Township Board of Education does hereby appoint Arthi Krishnan Grade 1 Teacher Mount Prospect School at a salary of Step 1-4 MA $59,355 effective January 19, 2021 through June 30, 2021. Salary to be prorated to reflect start date. Certificate of Eligibility-Alternate Route Mentoring required. 17) The Bernards Township Board of Education does hereby appoint Alexa McCaffrey Speech Teacher Mount Prospect School at a salary of Step 5 MA $60,200 effective February 1, 2021 through June 30, 2021. Salary to be prorated to reflect start date. 18) The Bernards Township Board of Education does hereby appoint Jennifer Walter Grade 1 Teacher Oak Street School at a salary of Step 1-4 BA $52,872 effective January 26, 2021 throu